<img src="https://drive.google.com/uc?export=view&id=1AH7OKu1s0FsJA3Dj3R9Z37I4gi0eb2Jq" width="100%">

# **Entrega 3**
---

## **0. Equipo de trabajo**
---

### **0.1. Integrantes del equipo de trabajo**


<table><thead>

  <tr>
    <td>1</td>
    <td>Cristhian Enrique Córdoba Trillos</td>
    <td>1030649666</td>
  </tr>
</tbody>
</table>

## **1. Evaluación de modelos**
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow import keras
import xgboost as xgb
import lightgbm as lgb
import warnings
import kagglehub
from kagglehub import KaggleDatasetAdapter
warnings.filterwarnings('ignore')
print("Librerías cargadas exitosamente")

Librerías cargadas exitosamente


In [2]:
!pip install -q gdown
import gdown
file_id = "1ti4-vA4cQw9s9vU3mvEibsYHwOjGeR5U"
output = "fraud_df.csv"
# Descargar
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)
# Cargarlo
import pandas as pd
df = pd.read_csv(output)

Downloading...
From: https://drive.google.com/uc?id=1ti4-vA4cQw9s9vU3mvEibsYHwOjGeR5U
To: /content/fraud_df.csv
100%|██████████| 50.0M/50.0M [00:00<00:00, 157MB/s]


In [3]:
pd.set_option('display.max_columns', None)
categorical_columns = ['category', 'gender', 'state', 'job', 'city', 'merchant']
numerical_columns = ['amt', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long',
                     'day', 'month', 'hour', 'minute', 'age']
df.head(2)

,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,day,month,year,hour,minute,year_dob,age,grupo_edad
0,5359543825610251,"fraud_Jenkins, Hauck and Friesen",gas_transport,59.91,Michael,Francis,M,1833 Jeanette Stravenue,Belgrade,MT,59714,45.7801,-111.1439,18182,"Engineer, drilling",1975-06-29,784eb609215cbaf3725642a7a9f8bb57,1379488419,45.274075,-111.649432,0,18,9,2020,7,13,1975,45,40-49
1,5540636818935089,fraud_Jast-McDermott,shopping_pos,3.96,Kenneth,Foster,M,329 Michael Extension,Lawrence,MA,1843,42.6911,-71.1605,76383,Geoscientist,1985-04-04,74e0ceb1e273a4a37f56dd2b5e6ce03d,1386265705,43.356278,-71.008959,0,5,12,2020,17,48,1985,35,30-39


In [4]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
#Aplicamos SMOTE
X_combo = df[categorical_columns + numerical_columns].copy()
y_combo = df['is_fraud']
X_combo_encoded = X_combo.copy()
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    X_combo_encoded[col] = le.fit_transform(X_combo[col])
    label_encoders[col] = le

# Crear pipeline de balanceo
over = SMOTE(sampling_strategy=0.1, random_state=42)  # 10% fraude
under = RandomUnderSampler(sampling_strategy=0.2, random_state=42)  # 20% fraude final
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# Aplicar pipeline
X_balanced, y_balanced = pipeline.fit_resample(X_combo_encoded, y_combo)

print(f"Forma original: {X_combo_encoded.shape}")
print(f"Forma balanceada: {X_balanced.shape}")
print(f"Balance final: {y_balanced.mean()*100:.2f}% fraude")

# Crear DataFrame final balanceado
df_final_balanced = pd.DataFrame(X_balanced, columns=X_combo_encoded.columns)
df_final_balanced['is_fraud'] = y_balanced

# Decodificar variables categóricas
for col, le in label_encoders.items():
    try:
        df_final_balanced[col] = le.inverse_transform(df_final_balanced[col].round().astype(int))
    except:
        # Si hay problemas con valores fuera de rango, mantener como numéricos
        pass

Forma original: (185239, 17)
Forma balanceada: (110544, 17)
Balance final: 16.67% fraude


In [5]:
X = df_final_balanced[categorical_columns + numerical_columns]
y = df_final_balanced['is_fraud']
# Codificación one-hot
X_encoded = pd.get_dummies(X, columns=categorical_columns, prefix=categorical_columns)

In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
# Escalar características numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
# Convertir a DF
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
print("Datos divididos y escalados")
print(f"Entrenamiento: {X_train_scaled.shape}")
print(f"Validación: {X_val_scaled.shape}")
print(f"Prueba: {X_test_scaled.shape}")

Datos divididos y escalados
Entrenamiento: (77380, 2143)
Validación: (16582, 2143)
Prueba: (16582, 2143)


In [7]:
df = df_final_balanced.copy()
categorical_columns = ['category', 'gender', 'state', 'job', 'city', 'merchant']
numerical_columns = ['amt', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long',
                     'day', 'month', 'hour', 'minute', 'age']
X = df[categorical_columns + numerical_columns]
y = df['is_fraud']

In [8]:
X_encoded = pd.get_dummies(X, columns=categorical_columns, prefix=categorical_columns)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
#Escalar
# numéricas
numeric_columns_for_scaling = [col for col in numerical_columns if col in X_train.columns]
scaler = StandardScaler()
#Train
X_train_scaled_df = X_train.copy()
X_train_scaled_df[numeric_columns_for_scaling] = scaler.fit_transform(X_train[numeric_columns_for_scaling])
# Test
X_val_scaled_df = X_val.copy()
X_val_scaled_df[numeric_columns_for_scaling] = scaler.transform(X_val[numeric_columns_for_scaling])
# ooo
X_test_scaled_df = X_test.copy()
X_test_scaled_df[numeric_columns_for_scaling] = scaler.transform(X_test[numeric_columns_for_scaling])

In [10]:
#  REGRESIÓN LOGÍSTICA CON ESCALADOS
lr_model = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced', C=1.0)
# Entrenar con datos de entrenamiento escalados
lr_model.fit(X_train_scaled_df, y_train)
# Validar con datos de validación
val_pred = lr_model.predict(X_val_scaled_df)
val_recall = recall_score(y_val, val_pred, zero_division=0)
val_precision = precision_score(y_val, val_pred, zero_division=0)
print(f"   Modelo validado:")
print(f"   Recall (validación): {val_recall:.4f}")
print(f"   Precision (validación): {val_precision:.4f}")

   Modelo validado:
   Recall (validación): 0.9106
   Precision (validación): 0.8451


In [11]:
# EVALUAR CON DATOS DE PRUEBA
print("EVALUACIÓN FINAL CON DATOS DE PRUEBA")
# Predicciones finales
y_pred = lr_model.predict(X_test_scaled_df)
y_pred_proba = lr_model.predict_proba(X_test_scaled_df)[:, 1]
# Métricas finales
accuracy = np.mean(y_pred == y_test)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"ROC AUC:   {roc_auc:.4f}")

EVALUACIÓN FINAL CON DATOS DE PRUEBA
Accuracy:  0.9523
Precision: 0.8296
Recall:    0.8985
F1-Score:  0.8627
ROC AUC:   0.9749


* El recall es del 0.89, es aceptable según nuestra definición inicial del proyecto.

In [12]:
# VALIDACIÓN CRUZADA
from sklearn.model_selection import cross_val_score, StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# Métricas
scoring_metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
#
cv_results = {}
cv_std = {}

print(" Resultados de Validación Cruzada (5-Fold):")
print("-" * 50)

for metric in scoring_metrics:
    try:
        scores = cross_val_score(lr_model, X_train_scaled_df, y_train,
                               cv=cv, scoring=metric, n_jobs=-1)
        cv_results[metric] = scores.mean()
        cv_std[metric] = scores.std()
        print(f"{metric.capitalize():>12}: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")
    except Exception as e:
        print(f"{metric.capitalize():>12}: Error - {e}")

print("-" * 50)

 Resultados de Validación Cruzada (5-Fold):
--------------------------------------------------
    Accuracy: 0.9530 (+/- 0.0019)
   Precision: 0.8352 (+/- 0.0099)
      Recall: 0.8943 (+/- 0.0071)
          F1: 0.8637 (+/- 0.0048)
     Roc_auc: 0.9719 (+/- 0.0004)
--------------------------------------------------


In [13]:
# ESTABILIDAD DEL MODELO
# Calcular coeficiente de variación para cada métrica
print("Coeficiente de variación (CV = std/mean):")
print("-" * 40)
stability_analysis = {}
for metric in scoring_metrics:
    if metric in cv_results and cv_results[metric] != 0:
        # Recalcular scores para obtener std
        scores = cross_val_score(lr_model, X_train_scaled_df, y_train,
                               cv=cv, scoring=metric, n_jobs=-1)
        mean_score = scores.mean()
        std_score = scores.std()

        if mean_score != 0:
            cv_ratio = std_score / mean_score
            stability_analysis[metric] = cv_ratio
            print(f"{metric.capitalize():<12}: {cv_ratio:.4f}")
        else:
            print(f"{metric.capitalize():<12}: Inf (mean = 0)")

# Interpretación simple
print("\n Interpretación de Estabilidad:")
print("-" * 30)
for metric, cv_ratio in stability_analysis.items():
    if cv_ratio < 0.1:
        stability = "1. Muy estable"
    elif cv_ratio < 0.2:
        stability = "2. Moderadamente estable"
    else:
        stability = "3. Poco estable"
    print(f"{metric.capitalize():<12}: {stability}")

Coeficiente de variación (CV = std/mean):
----------------------------------------
Accuracy    : 0.0010
Precision   : 0.0059
Recall      : 0.0040
F1          : 0.0028
Roc_auc     : 0.0002

 Interpretación de Estabilidad:
------------------------------
Accuracy    : 1. Muy estable
Precision   : 1. Muy estable
Recall      : 1. Muy estable
F1          : 1. Muy estable
Roc_auc     : 1. Muy estable


In [14]:
print("="*60)
# Resumen comparativo
print(f"{'Métrica':<12} {'Holdout':<10} {'CV Mean':<10} {'CV Std':<10} {'Diferencia':<12}")
print("-" * 60)

for metric in scoring_metrics:
    holdout_score = (
        accuracy if metric == 'accuracy' else
        precision if metric == 'precision' else
        recall if metric == 'recall' else
        f1 if metric == 'f1' else
        roc_auc
    )

    if metric in cv_results:
        cv_mean = cv_results[metric]
        cv_std_val = cv_std[metric]
        diff = abs(holdout_score - cv_mean)

        print(f"{metric.capitalize():<12} "
              f"{holdout_score:<10.4f} "
              f"{cv_mean:<10.4f} "
              f"{cv_std_val:<10.4f} "
              f"{diff:<12.4f}")

print("-" * 60)

Métrica      Holdout    CV Mean    CV Std     Diferencia  
------------------------------------------------------------
Accuracy     0.9523     0.9530     0.0009     0.0006      
Precision    0.8296     0.8352     0.0049     0.0056      
Recall       0.8985     0.8943     0.0035     0.0042      
F1           0.8627     0.8637     0.0024     0.0010      
Roc_auc      0.9749     0.9719     0.0002     0.0030      
------------------------------------------------------------


In [17]:
# GUARDAR RESULTADOS SIMPLES DE VALIDACIÓN CRUZADA
print("GUARDANDO RESULTADOS SIMPLES")
print("="*40)

# Crear diccionario simple con resultados
simple_cv_summary = {
    'validation_method': '5-Fold Stratified Cross Validation',
    'results': {},
    'stability': {}
}

for metric in scoring_metrics:
    if metric in cv_results:
        simple_cv_summary['results'][metric] = {
            'holdout': float(
                accuracy if metric == 'accuracy' else
                precision if metric == 'precision' else
                recall if metric == 'recall' else
                f1 if metric == 'f1' else
                roc_auc
            ),
            'cv_mean': float(cv_results[metric]),
            'cv_std': float(cv_std[metric])
        }

        # Análisis de estabilidad
        if cv_results[metric] != 0:
            cv_ratio = cv_std[metric] / cv_results[metric]
            simple_cv_summary['stability'][metric] = float(cv_ratio)

# Guardar resultados
import json
with open('/content/resultados_validacion_simple.json', 'w') as f:
    json.dump(simple_cv_summary, f, indent=2)

print("Resultados de validación cruzada simple guardados")
print("Archivo: resultados_validacion_simple.json")

# Mostrar conclusión final
print(f"\n CONCLUSIÓN FINAL:")
print(f"   Recall Holdout: {recall:.4f}")
print(f"   Recall CV Mean: {simple_cv_summary['results']['recall']['cv_mean']:.4f}")
print(f"   Diferencia: {abs(recall - simple_cv_summary['results']['recall']['cv_mean']):.4f}")
if abs(recall - simple_cv_summary['results']['recall']['cv_mean']) < 0.05:
    print(" + Buena consistencia entre holdout y validación cruzada")
else:
    print(" + Revisar")

GUARDANDO RESULTADOS SIMPLES
Resultados de validación cruzada simple guardados
Archivo: resultados_validacion_simple.json

CONCLUSIÓN FINAL:
   Recall Holdout: 0.8985
   Recall CV Mean: 0.8943
   Diferencia: 0.0042
 + Buena consistencia entre holdout y validación cruzada


## Guardar el modelo y componentes

In [18]:
print("GUARDANDO COMPONENTES PARA DESPLIEGUE")
print("="*50)
import joblib
import json
import os

os.makedirs('modelo', exist_ok=True)
# Guardar modelo entrenado
joblib.dump(lr_model, 'modelo/modelo_fraude.pkl')
print("Modelo guardado: modelo/modelo_fraude.pkl")
# Guardar scaler
joblib.dump(scaler, 'modelo/scaler.pkl')
print("Scaler guardado: modelo/scaler.pkl")

# Guardar características
features_info = {
    'numeric_features': numeric_columns_for_scaling,
    'categorical_features': categorical_columns,
    'all_features': list(X_train_scaled_df.columns)
}

with open('modelo/features.json', 'w') as f:
    json.dump(features_info, f, indent=2)
print("Información de características guardada: modelo/features.json")

# Guardar el resumen de resultados
with open('modelo/metricas_finales.json', 'w') as f:
    metricas = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1),
        'roc_auc': float(roc_auc),
        'cv_recall_mean': float(simple_cv_summary['results']['recall']['cv_mean']),
        'cv_recall_std': float(simple_cv_summary['results']['recall']['cv_std'])
    }
    json.dump(metricas, f, indent=2)
print("Métricas finales guardadas: modelo/metricas_finales.json")

print("\n Estructura creada:")
print("   /modelo/")
print("   ├── modelo_fraude.pkl")
print("   ├── scaler.pkl")
print("   ├── features.json")
print("   └── metricas_finales.json")

print("\nTodo OK. Guardado para despliegue.")

GUARDANDO COMPONENTES PARA DESPLIEGUE
Modelo guardado: modelo/modelo_fraude.pkl
Scaler guardado: modelo/scaler.pkl
Información de características guardada: modelo/features.json
Métricas finales guardadas: modelo/metricas_finales.json

 Estructura creada:
   /modelo/
   ├── modelo_fraude.pkl
   ├── scaler.pkl
   ├── features.json
   └── metricas_finales.json

Todo OK. Guardado para despliegue.
